In [244]:
import pandas as pd
pd.set_option('display.max_rows', None)  # 显示所有行
pd.set_option('display.max_columns', None)  # 显示所有列

In [245]:
import pandas as pd

# 读入 CSV 文件
df = pd.read_csv('evaluate#.csv')

df = df.drop(columns=['pbid'])
df = df[df['ARG'] <= 100000]
# df = df[df['classcial'] <= 500]
# df = df[df['in_constraints_probs'] >= 99]
grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
    "ARG": 'mean',
    'in_constraints_probs': 'mean',
    'best_solution_probs': 'mean',
    'iteration_count':'mean',
    'classcial':'mean',
    'run_times':'mean',
})

## 分组并把组作为索引
pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=["ARG", 'best_solution_probs', 'classcial', 'run_times'])
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

method_order = ['HeaSolver', 'PenaltySolver', 'ChocoSolver', 'QtoSimplifyDiscardSegmentedSolver', 'QtoSimplifyDiscardSolver', 'QtoSimplifySolver', 'QtoSolver']
# method_order = ['ChocoSolver', 'QtoSolver', 'QtoSimplifySolver', 'QtoSimplifyDiscardSolver']
# method_order = ['ChocoSolver']
# method_order = ['QtoSimplifyDiscardSegmentedSolver']
pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([["ARG",'best_solution_probs', 'classcial','run_times'], method_order]))

pivot_df

ARG                            \
                              HeaSolver PenaltySolver ChocoSolver   
pkid variables constraints                                          
0    6         3              47.009022     51.434867    0.095949   
1    15        8             157.863780    119.204157    0.294871   
2    21        12            162.335417    119.435028    0.360775   
3    28        15            274.899127    194.480496    0.571255   
4    12        6             177.985535    132.972738    0.215230   
5    15        9             302.760310    226.202840    0.608376   
6    20        8             521.207211    343.638284    1.100469   
7    24        12            796.905222    611.960307    1.530496   
8    8         6             354.850046    346.276477    0.319811   
9    15        8             416.912298    371.135567    0.945742   
10   18        9             621.042581    493.135243    1.391617   
11   21        10           1120.940823   1184.159265    3.501929   
12   7         4              65.804441     74.048259    0.063660   
13   10        5             194.259825    161.300335    0.422540   
14   14        6             193.995703    169.861083    0.538621   
15   18        7             403.311300    323.687016    0.744384   
16   9         4             943.826595    836.484974    3.936367   
17   12        5            1100.215629    958.824591    7.268241   
18   16        6            1423.962180   1256.712019   22.096429   
19   20        7            1615.093999   1512.362341   26.214772   

                                                              \
                           QtoSimplifyDiscardSegmentedSolver   
pkid variables constraints                                     
0    6         3                                    0.001673   
1    15        8                                    0.141654   
2    21        12                                   0.348680   
3    28        15                                   0.546348   
4    12        6                                    0.220680   
5    15        9                                    0.728246   
6    20        8                                    0.491267   
7    24        12                                   1.167991   
8    8         6                                    0.376480   
9    15        8                                    1.026056   
10   18        9                                    1.880657   
11   21        10                                   3.560862   
12   7         4                                    0.003787   
13   10        5                                    0.302639   
14   14        6                                    0.296263   
15   18        7                                    0.622732   
16   9         4                                    0.102136   
17   12        5                                    0.228372   
18   16        6                                    0.803507   
19   20        7                                    0.980151   

                                                                       \
                           QtoSimplifyDiscardSolver QtoSimplifySolver   
pkid variables constraints                                              
0    6         3                           0.008403          0.011123   
1    15        8                           0.115298          0.133791   
2    21        12                          0.138274          0.199832   
3    28        15                          0.526405          0.653707   
4    12        6                           0.146504          0.180615   
5    15        9                           0.433982          0.228525   
6    20        8                           0.457047          0.389243   
7    24        12                          0.933495          0.759606   
8    8         6                           0.066786          0.033675   
9    15        8                           0.887477          0.860032   
10   18        9          

In [246]:
# # 假设“variables”和“constraints”列作为多重索引列的一部分
# # 使用 .loc 选择多重索引列中含“variables”和“constraints”相关的列
# selected_columns = pivot_df.loc[:, pivot_df.columns.get_level_values(0).isin(['variables', 'constraints'])]

# selected_columns


In [247]:
# 选取 'depth' 下的所有列，并剔除 'QtoSolver ' 和 'QtoSimplifySolver'
df_1 = pivot_df['ARG'].drop(columns=['QtoSimplifyDiscardSolver', 'QtoSimplifySolver', 'QtoSolver'], errors='ignore')

# 选取 'culled_depth_per_param' 下的 'QtoSimplifyDiscardSolver' 列
df_2 = pivot_df['classcial'].drop(columns=['QtoSimplifyDiscardSolver', 'QtoSimplifySolver', 'QtoSolver'], errors='ignore')

# 合并处理后的 'depth' 列与 'QtoSimplifyDiscardSolver' 列，并指定后缀避免冲突
culled_depth_df = df_1.join(df_2, lsuffix='_1', rsuffix='_2')

# 按行遍历输出值，每个值用 & 分隔
for row in culled_depth_df.itertuples(index=False):
    formatted_row = [f"{round(x, 3):7.02f}" for x in row]
    print(" & ".join(formatted_row))


  47.01 &   51.44 &    0.10 &    0.00 &   25.88 &    9.34 &    1.03 &    0.54
 157.86 &  119.20 &    0.29 &    0.14 &  488.15 &  154.60 &    5.40 &  193.35
 162.34 &  119.44 &    0.36 &    0.35 &  775.12 &  263.46 &    9.94 &  296.13
 274.90 &  194.48 &    0.57 &    0.55 & 2483.38 &  425.57 &   26.19 &  734.88
 177.99 &  132.97 &    0.21 &    0.22 &  456.92 &  143.64 &    3.59 &    5.92
 302.76 &  226.20 &    0.61 &    0.73 &  610.38 &  208.62 &    2.73 &    1.63
 521.21 &  343.64 &    1.10 &    0.49 & 1112.50 &  305.94 &   21.43 &  413.41
 796.90 &  611.96 &    1.53 &    1.17 & 1046.06 & 1237.58 &   14.41 &  133.23
 354.85 &  346.28 &    0.32 &    0.38 &  106.75 &   37.33 &    1.97 &    1.23
 416.91 &  371.14 &    0.95 &    1.03 &  874.52 &  292.47 &    5.40 &    4.81
 621.04 &  493.13 &    1.39 &    1.88 & 1377.77 &  459.05 &   13.62 &    9.71
1120.94 & 1184.16 &    3.50 &    3.56 & 2383.61 &  675.43 &   19.59 &   16.13
  65.80 &   74.05 &    0.06 &    0.00 &   82.73 &   29.30 &    1

In [248]:
# # 从 pivot_df 中提取 variables 和 constraints 列
# variables_constraints_df = pivot_df.index.to_frame(index=False)[['variables', 'constraints']]
# variables_constraints_df

In [257]:
# 提取所需列
qto_arg = pivot_df['ARG']['QtoSimplifyDiscardSegmentedSolver']
choco_arg = pivot_df['ARG']['HeaSolver']
# qto_class = pivot_df['classcial']['QtoSimplifyDiscardSegmentedSolver']
# choco_class = pivot_df['classcial']['ChocoSolver']

# 方法1：每行相除取平均
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的倍数
row_wise_ratio_best_solution_probs = (choco_arg / qto_arg).mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的倍数
# row_wise_ratio_arg = (qto_class / choco_class).mean()

# 方法2：整列取平均后相除
# best_solution_probs 下 QtoSimplifyDiscardSolver / ChocoSolver 的整列平均的倍数
col_avg_ratio_best_solution_probs = choco_arg.mean() / qto_arg.mean()

# ARG 下 ChocoSolver / QtoSimplifyDiscardSolver 的整列平均的倍数
# col_avg_ratio_arg = qto_class.mean() / choco_class.mean()

# 输出结果
print(f"行平均 - arg: {row_wise_ratio_best_solution_probs:.2f}")
# print(f"行平均 - classcial: {row_wise_ratio_arg:.2f}")
print(f"列平均 - arg: {col_avg_ratio_best_solution_probs:.2f}")
# print(f"列平均 - classcial: {col_avg_ratio_arg:.2f}")


行平均 - arg: 3597.10
列平均 - arg: 787.78


In [250]:

# # 读入 CSV 文件
# df = pd.read_csv('evaluate_TSP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [251]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_SCP.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })

# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])

# method_order = ['PenaltySolver', 'CyclicSolver', 'HeaSolver', 'ChocoSolver']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [252]:
# 定义格式化函数
import numpy as np
def format_value(val):
    if np.isnan(val):  # 如果是NaN值则返回空字符
        return "NaN"
    elif val < 100:  # 如果值小于100，保留3位有效数字
        return f"{val:.2f}"
    else:  # 如果值大于等于100，不保留小数点
        return f"{val:.0f}"

# 按行遍历输出，每个值用 & 分隔
for row in pivot_df.itertuples(index=False):
    formatted_row = [format_value(x) for x in row]
    print(" & ".join(formatted_row))

47.01 & 51.43 & 0.10 & 0.00 & 0.01 & 0.01 & 0.01 & 4.75 & 5.66 & 88.34 & 99.67 & 96.69 & 96.52 & 97.75 & 25.88 & 9.34 & 1.03 & 0.54 & 0.41 & 0.69 & 0.69 & 331 & 126 & 119 & 91.80 & 69.52 & 136 & 137
158 & 119 & 0.29 & 0.14 & 0.12 & 0.13 & 0.14 & 0.00 & 0.12 & 37.72 & 72.36 & 67.70 & 71.39 & 71.08 & 488 & 155 & 5.40 & 193 & 12.89 & 22.20 & 23.07 & 331 & 116 & 121 & 63366 & 463 & 453 & 453
162 & 119 & 0.36 & 0.35 & 0.14 & 0.20 & 0.20 & 0.00 & 0.01 & 23.23 & 26.31 & 60.34 & 51.90 & 53.36 & 775 & 263 & 9.94 & 296 & 36.20 & 46.01 & 48.62 & 331 & 118 & 124 & 439326 & 546 & 453 & 453
275 & 194 & 0.57 & 0.55 & 0.53 & 0.65 & 0.62 & 0.00 & 0.00 & 15.75 & 15.24 & 21.55 & 13.14 & 15.69 & 2483 & 426 & 26.19 & 735 & 166 & 189 & 204 & 331 & 116 & 121 & 1405475 & 655 & 453 & 453
178 & 133 & 0.22 & 0.22 & 0.15 & 0.18 & 0.18 & 0.14 & 0.65 & 54.85 & 50.95 & 64.12 & 55.90 & 55.91 & 457 & 144 & 3.59 & 5.92 & 1.21 & 5.50 & 5.50 & 331 & 124 & 118 & 1687 & 150 & 432 & 431
303 & 226 & 0.61 & 0.73 & 0.43 & 0.23

In [253]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eva_old.csv')

# df = df.drop(columns=['pbid'])

# grouped_df = df.groupby(['pkid', 'layers', 'variables', 'constraints', 'method'], as_index=False).agg({
#     "ARG": 'mean',
#     'in_constraints_probs': 'mean',
#     'best_solution_probs': 'mean',
#     'iteration_count':'mean'
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid','variables','constraints'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df

In [254]:
pivot_df['ARG']

,,,HeaSolver,PenaltySolver,ChocoSolver,QtoSimplifyDiscardSegmentedSolver,QtoSimplifyDiscardSolver,QtoSimplifySolver,QtoSolver
pkid,variables,constraints,,,,,,,
0,6,3,47.009022,51.434867,0.095949,0.001673,0.008403,0.011123,0.006966
1,15,8,157.863780,119.204157,0.294871,0.141654,0.115298,0.133791,0.136345
2,21,12,162.335417,119.435028,0.360775,0.348680,0.138274,0.199832,0.198659
3,28,15,274.899127,194.480496,0.571255,0.546348,0.526405,0.653707,0.618630
4,12,6,177.985535,132.972738,0.215230,0.220680,0.146504,0.180615,0.180411
5,15,9,302.760310,226.202840,0.608376,0.728246,0.433982,0.228525,0.387764
6,20,8,521.207211,343.638284,1.100469,0.491267,0.457047,0.389243,0.396682
7,24,12,796.905222,611.960307,1.530496,1.167991,0.933495,0.759606,1.058158
8,8,6,354.850046,346.276477,0.319811,0.376480,0.066786,0.033675,0.043482


In [255]:
# import pandas as pd

# # 读入 CSV 文件
# df = pd.read_csv('eval_eliminate copy 2.csv')

# df = df.drop(columns=['pbid','layers', 'variables', 'constraints'])

# gr    ouped_df = df.groupby(['pkid',  'method'], as_index=False).agg({
#     "successrate": 'mean',
# })
# grouped_df
# # 假设grouped_df是一个Pandas DataFrame
# # grouped_df = grouped_df[((grouped_df['layers'] == 1) & (grouped_df['method'] == 'commute')) | ((grouped_df['layers'] == 7) & (grouped_df['method'] != 'commute'))]
# # grouped_df = grouped_df.drop(columns=['layers'])
# ## 分组并把组作为索引
# pivot_df = grouped_df.pivot(index =['pkid'], columns='method', values=['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'])




# method_order = ['penalty', 'cyclic', 'HEA', 'commute']
# pivot_df = pivot_df.reindex(columns=pd.MultiIndex.from_product([['best_solution_probs','in_constraints_probs',"ARG",'iteration_count'], method_order]))

# pivot_df